### Предсказывавем качество вина по его характеристикам

In [10]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
from sklearn import linear_model
from sklearn import tree # импортируем бибилиотеку дерево решений
from sklearn import metrics #метрики
from sklearn import ensemble #ансамбли
from sklearn.model_selection import train_test_split # делитель на тренировочную и тестовую выборки

import warnings # для игнорирования предупреждений
#Игнорируем предупреждения
warnings.filterwarnings('ignore')


# Устанавливаем стиль визуализаций в matplotlib
%matplotlib inline
plt.style.use('seaborn-v0_8')

Условно разделим вино на хорошее и плохое. Хорошим вином будем называть вино, параметр quality которого  6, плохим — всё остальное.

In [5]:
# считываем файл
wine_data = pd.read_csv('data/winequality-red.csv', sep=';')
wine_data.head(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5


In [6]:
# Преобразуем целевой признак в класс 0 и 1, где quality=1 если, >=6
wine_data['quality'] = wine_data['quality'].apply(lambda x: 1 if x >= 6 else 0)
wine_data.head(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0


In [7]:
# разделим дынные на матрицу Х и целевой признак у
X = wine_data.drop('quality', axis=1)
y = wine_data['quality']

Разделим выборку на тренировочную и тестовую

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Обучим модель на тренировочной выборке (X_train, y_train) модель случайного леса, состоящего из 500 деревьев. Максимальная глубина каждого из деревьев — 8. Минимальное число объектов в листе — 10. В качестве критерия информативности используйте критерий Джини. Количество признаков, на которых обучается каждое из деревьев, возьмите как $sqrt(m)$, где $m$ — общее количество признаков. В качестве значения параметра random_state возьмите число 42.

Сделаем предсказание качества вина для тренировочного и тестового наборов данных.
<br>Рассчитаем метрику accuracy для каждого набора с точностью до двух знаков после точки-разделителя

In [14]:
# создаем объект класса RandomTreeClassifeir
rf_clf = ensemble.RandomForestClassifier(
    n_estimators=500, #число деревьев
    criterion='gini', #критерий эффективности
    max_depth=8, #глубина дерева
    min_samples_leaf=10,
    max_features='sqrt', #число признаков из метода случайных подпространств
    random_state = 42
)

# Обучаем модель на тренировочной выборке
rf_clf.fit(X_train, y_train)

# Делаем предсказание класса на тренировочной выборке
y_train_pred = rf_clf.predict(X_train)

# выводим отчет о метриках тренировочной выборки
print(metrics.classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.82      0.86      0.84       603
           1       0.87      0.84      0.85       676

    accuracy                           0.85      1279
   macro avg       0.85      0.85      0.85      1279
weighted avg       0.85      0.85      0.85      1279



In [15]:
# Делаем предсказание класса на тестовой выборке
y_test_pred = rf_clf.predict(X_test)

# выводим отчет о метриках тестовой выборки
print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.70      0.74      0.72       141
           1       0.79      0.75      0.77       179

    accuracy                           0.75       320
   macro avg       0.75      0.75      0.75       320
weighted avg       0.75      0.75      0.75       320

